## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Benghazi,LY,32.1167,20.0667,65.03,66,0,10.20,clear sky
1,1,Severo-Kurilsk,RU,50.6789,156.1250,43.57,86,100,10.67,overcast clouds
2,2,Bloemhof,ZA,-27.6469,25.6070,49.24,80,96,10.51,light rain
3,3,Codrington,AU,-38.2667,141.9667,51.98,73,7,7.38,clear sky
4,4,Port Elizabeth,ZA,-33.9180,25.5701,56.32,99,100,6.91,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
6,6,Hilo,US,19.7297,-155.0900,89.28,77,75,13.80,broken clouds
8,8,Kasrawad,IN,22.1333,75.6000,83.91,58,0,18.48,clear sky
9,9,Sibolga,ID,1.7427,98.7792,77.40,88,81,2.48,broken clouds
10,10,Atuona,PF,-9.8000,-139.0333,77.54,66,0,9.35,clear sky
13,13,Cayenne,GF,4.9333,-52.3333,75.09,1,20,0.00,few clouds
14,14,Banda Aceh,ID,5.5577,95.3222,81.00,80,100,10.63,overcast clouds
15,15,Victoria,HK,22.2855,114.1577,80.96,79,36,3.00,scattered clouds
17,17,Forrest City,US,35.0081,-90.7898,83.88,55,0,15.66,clear sky
27,27,Bambanglipuro,ID,-7.9500,110.2833,77.23,89,100,2.06,overcast clouds
29,29,Faanui,PF,-16.4833,-151.7500,80.37,70,17,6.85,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        214
City           214
Country        214
Lat            214
Lng            214
Max Temp       214
Humidity       214
Cloudiness     214
Wind Speed     214
Description    214
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Not applicable since DataFrame has no empty rows 

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Hilo,US,89.28,broken clouds,19.7297,-155.0900,
8,Kasrawad,IN,83.91,clear sky,22.1333,75.6000,
9,Sibolga,ID,77.40,broken clouds,1.7427,98.7792,
10,Atuona,PF,77.54,clear sky,-9.8000,-139.0333,
13,Cayenne,GF,75.09,few clouds,4.9333,-52.3333,
14,Banda Aceh,ID,81.00,overcast clouds,5.5577,95.3222,
15,Victoria,HK,80.96,scattered clouds,22.2855,114.1577,
17,Forrest City,US,83.88,clear sky,35.0081,-90.7898,
27,Bambanglipuro,ID,77.23,overcast clouds,-7.9500,110.2833,
29,Faanui,PF,80.37,light rain,-16.4833,-151.7500,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Hilo,US,89.28,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Kasrawad,IN,83.91,clear sky,22.1333,75.6000,Hotel Aashray
9,Sibolga,ID,77.40,broken clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
10,Atuona,PF,77.54,clear sky,-9.8000,-139.0333,Villa Enata
13,Cayenne,GF,75.09,few clouds,4.9333,-52.3333,Hôtel Le Dronmi
...,...,...,...,...,...,...,...
676,Kirakira,SB,84.40,broken clouds,-10.4544,161.9205,SanBiz Lodge
677,Bilma,NE,88.00,overcast clouds,18.6853,12.9164,Expeditions Ténére voyages
680,Itarema,BR,76.87,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
692,Biak,ID,79.00,overcast clouds,-0.9131,122.8766,Penginapan Kim


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))